# Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

# Downloading CityScapes dataset from /www.cityscapes-dataset.com/

In [ ]:
!wget --keep-session-cookies --save-cookies=cookies.txt --post-data 'username=amrkandil&password=vukqYm-nyscuq-5rubfy&submit=Login' https://www.cityscapes-dataset.com/login/

!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=1

!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=3

## unzipping the files

In [ ]:
!unzip gtFine_trainvaltest.zip
!unzip leftImg8bit_trainvaltest.zip

# will ask you to replace the readme SO PRESS YES

# Loading cityscapes-to-coco-conversion repo

In [ ]:
!git clone https://github.com/kanndil/cityscapes-to-coco-conversion.git

In [ ]:

!pip install -r cityscapes-to-coco-conversion/requirements.txt 



# taking a small portion of the dataset

In [ ]:
# !rm -r gtFine/train/aachen
# !rm -r gtFine/train/bochum
# !rm -r gtFine/train/bremen
# !rm -r gtFine/train/cologne
# !rm -r gtFine/train/darmstadt
# !rm -r gtFine/train/dusseldorf
# !rm -r gtFine/train/erfurt
!rm -r gtFine/train/hamburg
# !rm -r gtFine/train/hanover
# !rm -r gtFine/train/jena
# !rm -r gtFine/train/krefeld
# !rm -r gtFine/train/monchengladbach
# !rm -r gtFine/train/strasbourg
# !rm -r gtFine/train/stuttgart
# !rm -r gtFine/train/tubingen
# !rm -r gtFine/train/ulm
# !rm -r gtFine/train/weimar

#  droppiing hamberg because it has a few corrupted images


# !rm -r leftImg8bit/train/aachen
# !rm -r leftImg8bit/train/bochum
# !rm -r leftImg8bit/train/bremen
# !rm -r leftImg8bit/train/cologne
# !rm -r leftImg8bit/train/darmstadt
# !rm -r leftImg8bit/train/dusseldorf
# !rm -r leftImg8bit/train/erfurt
!rm -r leftImg8bit/train/hamburg
# !rm -r leftImg8bit/train/hanover
# !rm -r leftImg8bit/train/jena
# !rm -r leftImg8bit/train/krefeld
# !rm -r leftImg8bit/train/monchengladbach
# !rm -r leftImg8bit/train/strasbourg
# !rm -r leftImg8bit/train/stuttgart
# !rm -r leftImg8bit/train/tubingen
# !rm -r leftImg8bit/train/ulm
# !rm -r leftImg8bit/train/weimar



In [ ]:
!mkdir data

In [ ]:
!mkdir data/cityscapes

In [ ]:
!mkdir data/cityscapes/annotations

In [ ]:
!mv leftImg8bit/ data/cityscapes/
!mv gtFine/ data/cityscapes/

# converting from cityscapes format to coco format

In [ ]:
!python cityscapes-to-coco-conversion/main.py --dataset cityscapes --datadir data/cityscapes --outdir data/cityscapes/annotations

# cloning DETR for Keras repo

In [ ]:
!rm -r detr-tensorflow
!rm -r detr_tf
!git clone https://github.com/kanndil/detr-tensorflow.git

In [ ]:
!mv detr-tensorflow/detr_tf/ ./

In [ ]:
# # !mkdir weights/
# # !mkdir weights/detr/
# import shutil
# shutil.copy("/content/drive/MyDrive/weights/detr/checkpoint" , "/content/weights/detr/")
# shutil.copy("/content/drive/MyDrive/weights/detr/detr.ckpt.data-00000-of-00001" , "/content/weights/detr/")
# shutil.copy("/content/drive/MyDrive/weights/detr/detr.ckpt.index" , "/content/weights/detr/")


'/content/weights/detr/detr.ckpt.index'

In [ ]:

import sys
# Set the path to the repository here
import detr_tf
import tensorflow as tf
from os.path import expanduser
import os
from detr_tf.data import load_coco_dataset
from detr_tf.training_config import TrainingConfig, DataConfig




In [ ]:

physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) == 1:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:

class MyConfig(TrainingConfig):
    def __init__(self):
        super().__init__()        
        self.data_dir = "./"
        self.data = DataConfig(data_dir=self.data_dir, img_dir="./data/cityscapes/", ann_file="./data/cityscapes/annotations/instancesonly_filtered_gtFine_train.json")
        self.batch_size = 1

config = MyConfig()

config2 = MyConfig()
config2.data =  DataConfig(data_dir=config2.data_dir, img_dir="./data/cityscapes/", ann_file="./data/cityscapes/annotations/instancesonly_filtered_gtFine_val.json")
# iterator, class_names = load_coco_dataset(config, config.batch_size, augmentation=True)







In [ ]:
train_iterator, class_names = load_coco_dataset(config, config.batch_size, augmentation=True)
valid_iterator, class_names = load_coco_dataset( config2, config.batch_size, augmentation=False)
print(class_names)

In [ ]:
from detr_tf.networks.detr import get_detr_model
# Load the pretrained DETR model with new heads at the top
# include_top: We do not include the last layers that predicts the bbox pos and class (include_top=False)
# nb_class: We add new layers on top of the model to predicts the bbox pos and class with three class (nb_class=3), background, helmet, face
# weights: Use the "detr" weight to init the model
detr = get_detr_model(config, include_top=False, nb_class=len(class_names), weights="detr")
detr.summary()

In [ ]:
# Train/finetune the transformers only
config.train_backbone = tf.Variable(False)
config.train_transformers = tf.Variable(False)
config.train_nlayers = tf.Variable(True)

In [ ]:
config.nlayers_lr = tf.Variable(1e-4)

In [ ]:
from detr_tf.optimizers import setup_optimizers
# Setup the optimziers and the trainable variables
optimzers = setup_optimizers(detr, config)

In [ ]:
!pip install wandb

In [ ]:
!wandb login 28bbc19edf27dceeec00dcae021c2d0421cedada

# you can connect it to your own wandb account follow https://docs.wandb.ai/quickstart

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Training the model

In [ ]:
# Train/finetune the transformers only
config.log = tf.Variable(True)
from detr_tf.optimizers import setup_optimizers
# Setup the optimziers and the trainable variables
optimzers = setup_optimizers(detr, config)

In [ ]:
from detr_tf import training

import wandb
if config.log:
    wandb.init(project="detr-tensorflow-with-nms", reinit=True)



In [ ]:
for epoch in range(1, 4):
    training.fit(detr, train_iterator, optimzers, config, epoch_nb=epoch, class_names=class_names)

# Visualizing the prediction for a random image from Validation 

In [ ]:
import tensorflow as tf
import numpy as np
import cv2


CLASS_COLOR_MAP = np.random.randint(0, 255, (100, 3))

from detr_tf import bbox

def numpy_bbox_to_image(image, bbox_list, labels=None, scores=None, class_name=[], config=None):
    """ Numpy function used to display the bbox (target or prediction)
    """
    assert(image.dtype == np.float32 and image.dtype == np.float32 and len(image.shape) == 3)

    if config is not None and config.normalized_method == "torch_resnet":
        channel_avg = np.array([0.485, 0.456, 0.406])
        channel_std = np.array([0.229, 0.224, 0.225])
        image = (image * channel_std) + channel_avg
        image = (image*255).astype(np.uint8)
    elif config is not None and config.normalized_method == "tf_resnet":
        image = image + mean
        image = image[..., ::-1]
        image = image  / 255
        
    bbox_xcycwh = bbox.np_rescale_bbox_xcycwh(bbox_list, (image.shape[0], image.shape[1])) 
    bbox_x1y1x2y2 = bbox.np_xcycwh_to_xy_min_xy_max(bbox_xcycwh)

    # Set the labels if not defined
    if labels is None: labels = np.zeros((bbox_x1y1x2y2.shape[0]))

    bbox_area = []
    # Go through each bbox
    for b in range(0, bbox_x1y1x2y2.shape[0]):
        x1, y1, x2, y2 = bbox_x1y1x2y2[b]
        bbox_area.append((x2-x1)*(y2-y1))

    # Go through each bbox
    for b in np.argsort(bbox_area)[::-1]:
        # Take a new color at reandon for this instance
        instance_color = np.random.randint(0, 255, (3))
        

        x1, y1, x2, y2 = bbox_x1y1x2y2[b]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        x1, y1, x2, y2 = max(0, x1), max(0, y1), min(image.shape[1], x2), min(image.shape[0], y2)

        # Select the class associated with this bbox
        class_id = labels[int(b)]

        if scores is not None and len(scores) > 0:
            label_name = class_name[int(class_id)]   
            label_name = "%s:%.2f" % (label_name, scores[b])
        else:
            label_name = class_name[int(class_id)]    

        class_color = CLASS_COLOR_MAP[int(class_id)]
    
        color = instance_color
        
        multiplier = image.shape[0] / 500
        cv2.rectangle(image, (x1, y1), (x1 + int(multiplier*7)*len(label_name), y1 + 10), class_color.tolist(), -5)
        cv2.putText(image, label_name, (x1+2, y1 + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4 * multiplier, (0, 0, 0), 1)
        cv2.rectangle(image, (x1, y1), (x2, y2), tuple(class_color.tolist()), 2)

    return image

In [ ]:
from detr_tf.inference import get_model_inference
#  numpy_bbox_to_image
import matplotlib.pyplot as plt
import numpy as np

for valid_images, target_bbox, target_class in valid_iterator:
    
    m_outputs = detr(valid_images, training=False)
    predicted_bbox, predicted_labels, predicted_scores = get_model_inference(m_outputs, config.background_class, bbox_format="xy_center")
    
    
    selected_indices = tf.image.non_max_suppression(
    predicted_bbox, predicted_scores, 15, 0.8)
    selected_boxes = tf.gather(predicted_bbox, selected_indices)
    selected_labels = tf.gather(predicted_labels, selected_indices)
    selected_scores = tf.gather(predicted_scores, selected_indices)
   

    result = numpy_bbox_to_image(
        np.array(valid_images[0]),
        np.array(selected_boxes),
        np.array(selected_labels),
        scores=np.array(selected_scores),
        class_name=class_names, 
        config=config
    )

    # result = numpy_bbox_to_image(
    #     np.array(valid_images[0]),
    #     np.array(predicted_bbox),
    #     np.array(predicted_labels),
    #     scores=np.array(predicted_scores),
    #     class_name=class_names, 
    #     config=config
    # )


    print(len(predicted_bbox))
    print(len(predicted_labels))
    
    print(len(selected_boxes))
    print(len(selected_labels))
    fig = plt.figure()
    fig.set_figheight(15)
    fig.set_figwidth(15)
    plt.imshow(result)
    plt.show()
    break

In [ ]:
config.train_transformers.assign(True)
config.transformers_lr.assign(1e-5)
config.nlayers_lr.assign(1e-4)

In [ ]:
for epoch in range(1, 15):
    training.fit(detr, train_iterator, optimzers, config, epoch_nb=epoch, class_names=class_names)

In [ ]:
from detr_tf.inference import get_model_inference
#  numpy_bbox_to_image
import matplotlib.pyplot as plt
import numpy as np

for valid_images, target_bbox, target_class in valid_iterator:
    
    m_outputs = detr(valid_images, training=False)
    predicted_bbox, predicted_labels, predicted_scores = get_model_inference(m_outputs, config.background_class, bbox_format="xy_center")
    
    
    selected_indices = tf.image.non_max_suppression(
    predicted_bbox, predicted_scores, 15, 0.8)
    selected_boxes = tf.gather(predicted_bbox, selected_indices)
    selected_labels = tf.gather(predicted_labels, selected_indices)
    selected_scores = tf.gather(predicted_scores, selected_indices)
   

    result = numpy_bbox_to_image(
        np.array(valid_images[0]),
        np.array(selected_boxes),
        np.array(selected_labels),
        scores=np.array(selected_scores),
        class_name=class_names, 
        config=config
    )

    # result = numpy_bbox_to_image(
    #     np.array(valid_images[0]),
    #     np.array(predicted_bbox),
    #     np.array(predicted_labels),
    #     scores=np.array(predicted_scores),
    #     class_name=class_names, 
    #     config=config
    # )


    print(len(predicted_bbox))
    print(len(predicted_labels))
    
    print(len(selected_boxes))
    print(len(selected_labels))
    fig = plt.figure()
    fig.set_figheight(15)
    fig.set_figwidth(15)
    plt.imshow(result)
    plt.show()
    break

# save weights

In [ ]:
!mkdir -p saved_weights/
!mkdir -p saved_weights/detr

detr.save_weights("saved_weights/detr/",True)



# setting up evaluation

In [ ]:
import argparse
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np

from detr_tf.inference import get_model_inference
from detr_tf.data.coco import load_coco_dataset
from detr_tf.loss.compute_map import cal_map, calc_map, APDataObject
from detr_tf.networks.detr import get_detr_model
from detr_tf.bbox import xcycwh_to_xy_min_xy_max, xcycwh_to_yx_min_yx_max
from detr_tf.inference import numpy_bbox_to_image
from detr_tf.training_config import TrainingConfig, training_config_parser


In [ ]:

def eval_model(model, config, class_names, valid_dt):
    """ Run evaluation
    """

    iou_thresholds = [x / 100. for x in range(50, 100, 5)]
    # print(iou_thresholds)
    ap_data = {
        'box' : [[APDataObject() for _ in class_names] for _ in iou_thresholds],
        'mask': [[APDataObject() for _ in class_names] for _ in iou_thresholds]
    }

    it = 0

    for images, target_bbox, target_class in valid_dt:
        # print("in new image ", it)
        # Forward pass
        m_outputs = model(images)
        # Run predictions
        p_bbox, p_labels, p_scores = get_model_inference(m_outputs, config.background_class, bbox_format="yxyx")
        # Remove padding
        t_bbox, t_class = target_bbox[0], target_class[0]
        size = tf.cast(t_bbox[0][0], tf.int32)
        t_bbox = tf.slice(t_bbox, [1, 0], [size, 4])
        t_bbox = xcycwh_to_yx_min_yx_max(t_bbox)
        t_class = tf.slice(t_class, [1, 0], [size, -1])
        t_class = tf.squeeze(t_class, axis=-1)
        # Compute map
        cal_map(p_bbox, p_labels, p_scores,  np.zeros((138, 138, len(p_bbox))), np.array(t_bbox), np.array(t_class), np.zeros((138, 138, len(t_bbox))), ap_data, iou_thresholds)
        # print("before f ")
        print(f"Computing map.....{it}", end="\r")
        it += 1
        #if it > 10:
        #    break

    # Compute the mAp over all thresholds
    calc_map(ap_data, iou_thresholds, class_names, print_result=True)

In [ ]:
from detr_tf import training
config.ann_file = tf.Variable("./data/cityscapes/annotations/instancesonly_filtered_gtFine_val.json")
for epoch in range(1, 100):
    training.eval(detr, valid_iterator, config, class_names, 2)